In [1]:
import pandas as pd
import numpy as np
!pip install sklearn
from sklearn.model_selection import train_test_split

Defaulting to user installation because normal site-packages is not writeable


In [2]:
df = pd.read_csv('ratings.csv')

In [3]:
df=df.loc[df['rating']>2]

In [4]:
df.loc[df['userId']==442]
df.drop([68317,68336], axis=0, inplace=True)

In [5]:
df.loc[df['userId']==508]
df.drop([80352,80353,80357,80358,80363,80364], axis=0, inplace=True)

In [6]:
df_1=df.loc[df['userId']==1]
train1, test1 = train_test_split(df_1, test_size=0.2)

In [7]:
df_train,df_test=train_test_split(df,test_size=0.2,stratify=df['userId'])

In [8]:
df.drop(columns=['rating','timestamp'],inplace=True)
merge_list = df_train.groupby(by = ['userId'])['movieId'].apply(list).reset_index()
merge_list = merge_list['movieId'].tolist()

In [9]:
merge_list2 = df_test.groupby(by = ['userId'])['movieId'].apply(list).reset_index()
merge_list2 = merge_list2['movieId'].tolist()
merge_list2

[[223,
  1136,
  3729,
  1060,
  2700,
  1196,
  1240,
  1030,
  157,
  6,
  3253,
  2018,
  1377,
  1210,
  3062,
  1089,
  2542,
  231,
  2993,
  3450,
  2761,
  943,
  1291,
  2139,
  3247,
  1408,
  1023,
  596,
  2105,
  2033,
  1226,
  733,
  2716,
  3744,
  110,
  648,
  3147,
  441,
  2944,
  1500,
  1198,
  101,
  1804,
  2948,
  3809],
 [80906, 58559, 115713, 6874, 106782, 79132],
 [5919, 5764, 2851, 70946],
 [215,
  1250,
  4381,
  1291,
  1225,
  919,
  21,
  3897,
  2858,
  342,
  1288,
  1517,
  1196,
  4765,
  1947,
  4021,
  2683,
  1885,
  1094,
  1086,
  2078,
  4741,
  1073,
  3033,
  608,
  1732,
  1266,
  2791,
  2692,
  2145,
  1080,
  2997,
  3255,
  3044],
 [150, 608, 247, 349, 110, 594, 290, 300],
 [835,
  366,
  500,
  539,
  472,
  4,
  377,
  171,
  347,
  524,
  205,
  141,
  783,
  151,
  256,
  458,
  490,
  274,
  360,
  542,
  587,
  302,
  493,
  5,
  747,
  1073,
  177,
  282,
  47,
  1084,
  450,
  60,
  11,
  279,
  548,
  189,
  489,
  100,
  552,


In [10]:
# from mlxtend.preprocessing import TransactionEncoder
# te = TransactionEncoder()
# te_ary = te.fit(merge_list).transform(merge_list)
# df = pd.DataFrame(te_ary, columns=te.columns_)

In [11]:
# df.head()

In [12]:
# from mlxtend.frequent_patterns import fpgrowth
# %time
# fpgrowth_frequent_itemsets = fpgrowth(df, min_support=0.05, use_colnames=True,max_len=2)
# fpgrowth_frequent_itemsets.head()

In [13]:
# fpgrowth_frequent_itemsets['itemsets'].apply(lambda x: len(x)).value_counts()

In [14]:
# fpgrowth_frequent_itemsets['length'] = fpgrowth_frequent_itemsets['itemsets'].apply(lambda x: len(x))
# fpgrowth_frequent_itemsets

In [15]:
# from mlxtend.frequent_patterns import association_rules
# rules = association_rules(fpgrowth_frequent_itemsets,metric="lift",min_threshold=0.01)
# rules

In [16]:
# rules[rules["antecedents"].apply(lambda x: '356' in str(x))].groupby(
#     ['antecedents', 'consequents'])[['confidence']].max().sort_values(ascending=False,
#                                                                       by='confidence').head(10).plot(kind='bar').invert_xaxis()


In [17]:
class node:
    def __init__(self, word, word_count=0, parent=None, link=None):
        self.word=word
        self.word_count=word_count
        self.parent=parent
        self.link=link
        self.children={}

#tree traversal
    def visittree(self):
#        if self is None:
#            return None
        output=[]
        output.append(str(vocabdic[self.word]) + " " +str(self.word_count))
        if len(list(self.children.keys()))>0:
            for i in (list(self.children.keys())):
                output.append(self.children[i].visittree())
        return output
  
              
'''      Build FPTREE class and method       '''        
class fptree:
    def __init__(self, data, minsup=400):
        #raw data and minminual support
        self.data=data
        self.minsup=minsup
        
        #null root
        self.root= node(word="Null", word_count=1)
        
        #each line of transaction with new order from the most frequent items to less
        self.wordlinesort=[]
        #node table containing link of all nodes of same word
        self.nodetable=[]
        #dictionary contaiing word more than the minsupport count with des order
        self.wordsortdic=[]
       
        #dictionaly containing word and the support count        
        self.worddic={}
        #dictionary with word and it's postion of the support count rank
        self.wordorderdic={}
#        
#        self.preprocess(data)
#        #first scan to build all the necessay dictionary
        self.construct(data)
        #second scan and build fp tree line  by line            
    def construct(self, data):
                #get support count for all word
        for tran in data:
            for words in tran:
                if words in self.worddic.keys():
                    self.worddic[words]+=1
                else:
                    self.worddic[words]=1
        wordlist = list(self.worddic.keys())
        #prune all the world with < min support count
        for word in wordlist:
            if(self.worddic[word]<self.minsup):
                del self.worddic[word]
        #sort the remaing items des, with first word count than work#id        
        self.wordsortdic = sorted(self.worddic.items(), key=lambda x: (-x[1],x[0])) 
        #create a table containing word, wordcount and all link node of that word
        t=0
        for i in self.wordsortdic:
            word = i[0]
            wordc = i[1]
            self.wordorderdic[word]=t
            t+=1
            wordinfo = {'wordn':word, 'wordcc':wordc, 'linknode': None}
            self.nodetable.append(wordinfo)
        #construct fptree line by line
    
        for line in data:
            supword=[]
            for word in line:
                #only keep words with support count higher than minsupport
                if word in self.worddic.keys():
                    supword.append(word)
           #insert words to the fp tree
            if len(supword)>0:
                #reorder the words 
                sortsupword = sorted(supword, key = lambda k: self.wordorderdic[k])
                self.wordlinesort.append(sortsupword)
                #enter the word one by one from begining
                R = self.root
#                print(sortsupword)
                for i in sortsupword:                  
                    if i in R.children.keys():
                        R.children[i].word_count +=1
                        R=R.children[i]
                    else:

                        R.children[i] = node(word=i,word_count=1,parent=R,link=None)
                        R=R.children[i]
                        # link this node to nodetable
                        for wordinfo in self.nodetable:
                            if wordinfo["wordn"] == R.word:
                                # find the last node of the  node linklist
                                if wordinfo["linknode"] is None:
                                    wordinfo["linknode"] = R
                                else:
                                    iter_node = wordinfo["linknode"]
                                    while(iter_node.link is not None):
                                        iter_node = iter_node.link
                                    iter_node.link = R

# create transactions for conditinal tree   
    def condtreetran(self,N):
        if N.parent is None:
            return None
        
        condtreeline =[]
        #starting from the leaf node reverse add word till hit root
        while N is not None:
            line=[]
            PN = N.parent
            while PN.parent is not None:
                line.append(PN.word)
                PN=PN.parent
            #reverse order the transaction
            line = line[::-1]
            for i in range(N.word_count):
                condtreeline.append(line)   
            #move on to next linknode
            N=N.link
        return condtreeline
    
#Find frequent word list by creating conditional tree
    def findfqt(self,parentnode=None):
        if len(list(self.root.children.keys()))==0:
            return None
        result=[]
        sup=self.minsup
        #starting from the end of nodetable
        revtable = self.nodetable[::-1]
        for n in revtable:
            fqset=[set(),0]
            if(parentnode==None):      
                fqset[0]={n['wordn'],}
            else:
                fqset[0] = {n['wordn']}.union(parentnode[0])
            fqset[1]=n['wordcc']
            result.append(fqset)
            condtran = self.condtreetran(n['linknode'])
            #recursively build the conditinal fp tree
            contree= fptree(condtran,sup)
            conwords = contree.findfqt(fqset)
            if conwords is not None:
                for words in conwords:
                    result.append(words)
        return result

#check if tree hight is larger than 1 
    def checkheight(self):
        if len(list(self.root.children.keys()))==0:
            return False
        else:
            return True

In [ ]:
min_sup=0.8
test_data=merge_list
fp_tree = fptree(test_data, min_sup)
print ("\n========== Printing Frequent Word Set on "+" ==========")
frequentwordset = fp_tree.findfqt() #mining frequent patt
frequentwordset=sorted(frequentwordset,key = lambda k: -k[1] )


#print frequent patt
for word in frequentwordset:
    count = (str(word[1])+"\t")
    words =''
    for val in word[0]:
        words+= (str(vocabdic[val])+" ")
    print(count+words)


#print conditional fp tree height >1
for i in fp_tree.nodetable[::-1]:
    lines = fp_tree.condtreetran(i['linknode'])
    condtree = fptree(lines,min_sup)
    if(condtree.checkheight()):
        print('Condtional FPTree Root on '+(vocabdic[i['wordn']]))
        print(condtree.root.visittree())


========== Printing Frequent Word Set on  ==========
